# Python Regular Expressions - Mostly Harmless

![Oh XKCD](https://imgs.xkcd.com/comics/perl_problems.png)

Regular expressions are a powerful tool with several valid uses.  Unfortunately, they are also a tool that people like to misuse, which is usually the root cause of the horror stories you may have heard.

If you haven't heard any horror stories about them... well... ignore that last paragraph.

Regular expressions are simply a search pattern; a special string that will either accept or reject another string.  As an abstract concept, regular expressions can be messy and difficult to work with.  We, however, will be using them for concrete tasks that are much easier to understand.

## The `re` module

Python's implementation of regular expressions are in the `re` module.  There is a "gentle" introduction found at https://docs.python.org/3/howto/regex.html#regex-howto, but we will take a much more focused (and probably easier) approach.

The first thing to realize is that our pattern we will be matching is a string.  We can use special characters to add flexibility to our patterns.  The Python RE Special characters are `. ^ $ * + ? { } [ ] \ | ( )`.  While we won't cover all of these, we will see how some of these can be super CoolThings(tm).

Let's investigate some of the `re` functions without using special characters.  We will talk about the functions and return types of these functions after working our way through using the special characters to do interesting things.

In [3]:
from re import search, match

# search for the string 'asdf' in the target
print(search('asdf', "I've got a loverly bunch of coconuts (do de do)"))
print(search('asdf', "I've got to sneezasdf"))
# See if the target string starts with 'asdf'
print(match('asdf', "I've got a loverly bunch of coconuts (do de do)"))
print(match('asdf', "I've got to sneezasdf"))

None
<_sre.SRE_Match object; span=(17, 21), match='asdf'>
None
None


Searching for a fixed string is not particularly useful (we could just use the `in` operator or `startswith`.  Where regular expressions get cool is the use of the special characters.

## Matching sets of characters: `[ ]`

Let's consider a real world problem.  Consider a totally hypothetical computer network where usernames are an upper or lower case `s` followed by six digits.  Writing a regular expression to determine if this hypothetical "s number" exists in a string as we did above would be... tricky (there are $2*10^6$ possible s numbers if we are not case sensitive, and writing each one out would be exhaustive).

We can define a set of characters such that the string is accepted if it contains **any** of the characters in the set.  For example `[sS]` would match either a lower or uppercase s.  `[1234567890]` would match any single digit.  Our s number example can be described as a lower or uppercase s followed by six digits, or:

In [5]:
from re import search, match

snum_regex = '[sS][1234567890][1234567890][1234567890][1234567890][1234567890][1234567890]'
print(search(snum_regex, 's123456'))
# still finds the pattern because the first seven characters are a valid s#
print(search(snum_regex, 's1234567'))
print(search(snum_regex, 'S829373'))
print(search(snum_regex, 'nathane'))

<_sre.SRE_Match object; span=(0, 7), match='s123456'>
<_sre.SRE_Match object; span=(0, 7), match='s123456'>
<_sre.SRE_Match object; span=(0, 7), match='S829373'>
None


Note that if you want to match any character _except_ what is in the set, you can start the set with the special character `^`; `[^asdf]` will match any any character except a, s, d, or f.  You can combine this with other special characters to get even more interesting pattern matching.

## Condensing and repetition

As established, I'm lazy, and the above is still VERY verbose.  One thing to realize is that the digits (`0` through `9`) are sequential in their ASCII/ordinal values.  Inside a set, we can use the `-` to tell it to match a range.  The regex `[0-9]` will match any single digit.  We can rewrite the above as:

In [6]:
from re import search, match

snum_regex = '[sS][0-9][0-9][0-9][0-9][0-9][0-9]'
print(search(snum_regex, 's123456'))
# still finds the pattern because the first seven characters are a valid s#
print(search(snum_regex, 's1234567'))
print(search(snum_regex, 'S829373'))
print(search(snum_regex, 'nathane'))

<_sre.SRE_Match object; span=(0, 7), match='s123456'>
<_sre.SRE_Match object; span=(0, 7), match='s123456'>
<_sre.SRE_Match object; span=(0, 7), match='S829373'>
None


Still too much effort, methinks.  Regular expressions allow us to specify the _number_ of times a character must appear using `{x}`, which means to repeat the last character acceptance pattern `x` times.  The regex `a{5}` will match the string `aaaaa`.  Using the magic of repetition specification, we can write an even more succinct regular expression:

In [7]:
from re import search, match

# match an upper or lower case s followed by 6 digits
snum_regex = '[sS][0-9]{6}'
print(search(snum_regex, 's123456'))
# still finds the pattern because the first seven characters are a valid s#
print(search(snum_regex, 's1234567'))
print(search(snum_regex, 'S829373'))
print(search(snum_regex, 'nathane'))

<_sre.SRE_Match object; span=(0, 7), match='s123456'>
<_sre.SRE_Match object; span=(0, 7), match='s123456'>
<_sre.SRE_Match object; span=(0, 7), match='S829373'>
None


But what if we don't know exactly the number of times we want to match something.  Let's switch our problem to another common one we've encountered: how can we tell if a string is an allowable variable or function name (also known as an identifier).  Consider the rules for most common identifiers:

* Can only contain letters, numbers, and underscores
* Must start with either a letter or an underscore

We can tell the regular expression to repeat the previous pattern for an indeterminite number of times.  The `+` and `*` characters mean to repeat the regular pattern: `+` means "repeat one or more times", and `*` means "repeat any number of times (including zero)".  The regular expression `[a-z]+` will match one or more lowercase letters.  Let's see if we can write a regular expression to match an identifier.

In [11]:
from re import search

# match any letter or underscore, followed by any number of letters, underscores, or numbers
id_regex = '[a-zA-Z_][a-zA-Z0-9_]*'

print(search(id_regex, 'valid_var_name'))
print(search(id_regex, '_')) # totally a valid variable name
print(search(id_regex, '2cool4roolz')) # why doesn't this fail?  look at the 'match' field of the result
print(search(id_regex,'__a_s_d_1_2'))
print(search(id_regex, '3'))

<_sre.SRE_Match object; span=(0, 14), match='valid_var_name'>
<_sre.SRE_Match object; span=(0, 1), match='_'>
<_sre.SRE_Match object; span=(1, 11), match='cool4roolz'>
<_sre.SRE_Match object; span=(0, 11), match='__a_s_d_1_2'>
None


## Other special characters

The other special characters have meaning; some are simple enough to warrant an easy definition, and others we will probably not use:

* `.` match literally any character (except newline)
* `?` match zero or one of the previous pattern (also used for other black magic we will not talk about
* `()` Group the enclosed patterns (allows repetition of multiple patterns or other interesting things)
* `|` In a group; let the group match either side (an "or")
* `\` Escape the next character (if you want to match a special character, for instance
* `^` When not in a character class (`[]`), match the beginning of a line/string
* `$` Match the end of the string/line

## A note on "compiling" regular expressions

Regular expressions are not necessarily fast in Python.  However, there is a utility that allows you to speed up the process (and simplify the function/method calls).

Python allows you to compile a regular expression.  This creates a regular expression object that has the same methods as the `re` module.

In [14]:
from re import search, match, compile



snum_regex = '[sS][0-9]{6}'
snum_re = compile(snum_regex)

print(search(snum_regex, 's123543'))
print(snum_re.search('s123543'))

print("RACE!")
from timeit import timeit
print('No compilation:', timeit("search(snum_regex, 's123543')", "from re import search; snum_regex = '[sS][0-9]{6}'"), 's/million')
print('No compilation:', timeit("snum_re.search('s123543')", "from re import search, compile; snum_regex = '[sS][0-9]{6}';snum_re = compile(snum_regex)"), 's/million')

<_sre.SRE_Match object; span=(0, 7), match='s123543'>
<_sre.SRE_Match object; span=(0, 7), match='s123543'>
RACE!
No compilation: 0.7165242050000415 s/million
No compilation: 0.2593930860000455 s/million


If you are only using a regular expression occasionally, it may not be worth compiling the regular expression.  One more benefit is that it allows you to associate a regular expression with a name, so your use of regular expressions is clearer in your source code.

## Character Classes

Python has defined some special character classes you can use:

* `\d`: any decimal digit (equivalent to `[0-9]`)
* `\D`: any non-digit character (equivalent to `[^0-9]`)
* `\s`: any whitespace character (equivalent to `[ \t\n\r\f\v]`)
* `\S`: any non-whitespace character (equivalent to `[^ \t\n\r\f\v]`)
* `\w`: any alphanumeric character (equivalent to `[a-zA-Z0-9_]`)
* `\w`: any non-alphanumeric character (equivalent to `[^a-zA-Z0-9_]`)

When using these in regular expressions, we should indicate that our string is a a "raw string" by putting an `r` before the first quote.  This tells python to not interpret escaped characters.  While not an issue for now, eventually a non-raw string that contains, say, `\w` (which is not a valid python escape sequence) will throw a syntax error.

Any of these characters can be used in our "custom" character classes (inside `[]`).  We can, for example, rewrite our previous regular expressions as:

In [19]:
import re

# Note the r before the quotes; this tells python it is a raw string
snum_re = re.compile(r'[sS]\d{6}')
ident_re = re.compile(r'[a-zA-Z_]\w*')
# An alternate way to do this by putting the "not" matching in the middle
# [\w^\d] means any alphanumeric character EXCEPT digis
ident_re_alt = re.compile(r'[\w^\d]\w*')

print(snum_re.search('s123456'))
print(snum_re.search('nathane'))
print(ident_re.search('_asdf'))
print(ident_re_alt.search('___2ad'))

<_sre.SRE_Match object; span=(0, 7), match='s123456'>
None
<_sre.SRE_Match object; span=(0, 5), match='_asdf'>
<_sre.SRE_Match object; span=(0, 6), match='___2ad'>


## Other useful `re` functionality

So far we have seen `match` (which determines if the regular expression matches at the beginning of the string) and `search`.  

These methods return a `Group` object that contains information about the matching portion of the string.  This object contains the following methods:

* `group()`: returns the string that match (useful when you are using patterns)
* `start()`: returns the starting index of the matched group
* `end()`: returns the ending index of the matched group
* `span()`: returns (start(), end()) as a tuple

Other helpful methods include `findall` and `finditer`, which return a list and an iterator of _all_ matching substrings in the string (where `match` and `search` only find one).

## Find/Replace

One _incredibly_ useful application of regular expressions is performign a find/replace operation (we will often use this to clean data and replace unhelpful/invalid characters (often whitespace).  The `sub(repl, string, count=0, flags=0)` method allows us to do pattern based find/replace.  Consider the following way to replace ALL whitespace in a string with underscores:

In [22]:
import re

ws = re.compile(r'\s')

# Multiline strings are fun!
# from https://www.reddit.com/r/Jokes/comments/1bkzx7/my_favourite_haiku/
text = '''Space is limited
In a haiku, so it's hard
To finish what you'''

print(ws.sub('_', text))

# alternately, we can use re.sub, remember that the pattern objects have the same
# methods as the module has functions
print(re.sub(r'\s', '_', text))

Space_is_limited_In_a_haiku,_so_it's_hard_To_finish_what_you
Space_is_limited_In_a_haiku,_so_it's_hard_To_finish_what_you


There are MANY other things you can do with regular expressions, but this is hopefully a good introduction.  You can always get more information in the "gentle" introduction (https://docs.python.org/3.7/howto/regex.html#regex-howto) as well as much more rigorous documentation in the official regex docs (https://docs.python.org/3.7/library/re.html).